## import library

In [1]:
import os
import numpy as np
import tensorflow as tf
from data_loader import *
from data_utils import *
from model import *
from const import *

c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\ten

# prepare data

In [2]:
train_data, test_data = getBagImage()

Reading csv................
Load bag image...................
Number of bag train data:  1987
Number of bag test data:  484
---------------------------------------------------------------


In [3]:
def one_hot(index, num_classes):
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

# Main Training code

## Input tensor + Model Network 

In [4]:
sess = tf.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()

x, y, mask = input_tensor()

y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv, is_training, keep_prob = multi_label_net(x)

W0820 11:28:26.786340  2396 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0820 11:28:26.787338  2396 deprecation.py:323] From <ipython-input-4-eaa24b9c7f9c>:2: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0820 11:28:26.792324  2396 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:7: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 11:28:26.795317  2396 deprecation_wrapper.py:119] From C:\Users\shimd\

## Loss + mask 씌우기

In [5]:
color_loss, shape_loss, opening_loss, strap_loss, pattern_loss, carrying_loss, charm_loss, decoration_loss, l2_loss, total_loss = selective_loss(y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv,y, mask)

train_step = train_op(total_loss, global_step)

color_mask = tf.get_collection('color_mask')[0]
shape_mask = tf.get_collection('shape_mask')[0]
opening_mask = tf.get_collection('opening_mask')[0]
strap_mask = tf.get_collection('strap_mask')[0]
pattern_mask = tf.get_collection('pattern_mask')[0]
carrying_mask = tf.get_collection('carrying_mask')[0]
charm_mask = tf.get_collection('charm_mask')[0]
decoration_mask = tf.get_collection('decoration_mask')[0]

y_color = tf.get_collection('y_color')[0]
y_shape = tf.get_collection('y_shape')[0]
y_opening = tf.get_collection('y_opening')[0]
y_strap = tf.get_collection('y_strap')[0]
y_pattern = tf.get_collection('y_pattern')[0]
y_carrying = tf.get_collection('y_carrying')[0]
y_charm = tf.get_collection('y_charm')[0]
y_decoration = tf.get_collection('y_decoration')[0]

color_correct_prediction = tf.equal(tf.argmax(y_color_conv, 1), tf.argmax(y_color, 1))
shape_correct_prediction = tf.equal(tf.argmax(y_shape_conv, 1), tf.argmax(y_shape, 1))
opening_correct_prediction = tf.equal(tf.argmax(y_opening_conv, 1), tf.argmax(y_opening, 1))
strap_correct_prediction = tf.equal(tf.argmax(y_strap_conv, 1), tf.argmax(y_strap, 1))
pattern_correct_prediction = tf.equal(tf.argmax(y_pattern_conv, 1), tf.argmax(y_pattern, 1))
carrying_correct_prediction = tf.equal(tf.argmax(y_carrying_conv, 1), tf.argmax(y_carrying, 1))
charm_correct_prediction = tf.equal(tf.argmax(y_charm_conv, 1), tf.argmax(y_charm, 1))
decoration_correct_prediction = tf.equal(tf.argmax(y_decoration_conv, 1), tf.argmax(y_decoration, 1))

color_true_pred = tf.reduce_sum(tf.cast(color_correct_prediction, dtype=tf.float32) * color_mask)
shape_true_pred = tf.reduce_sum(tf.cast(shape_correct_prediction, dtype=tf.float32) * shape_mask)
opening_true_pred = tf.reduce_sum(tf.cast(opening_correct_prediction, dtype=tf.float32) * opening_mask)
strap_true_pred = tf.reduce_sum(tf.cast(strap_correct_prediction, dtype=tf.float32) * strap_mask)
pattern_true_pred = tf.reduce_sum(tf.cast(pattern_correct_prediction, dtype=tf.float32) * pattern_mask)
carrying_true_pred = tf.reduce_sum(tf.cast(carrying_correct_prediction, dtype=tf.float32) * carrying_mask)
charm_true_pred = tf.reduce_sum(tf.cast(charm_correct_prediction, dtype=tf.float32) * charm_mask)
decoration_true_pred = tf.reduce_sum(tf.cast(decoration_correct_prediction, dtype=tf.float32) * decoration_mask)

real_train_data = []

# Mask : color -> 0 , shape -> 1, opening -> 2, strap -> 3, pattern -> 4, carrying -> 5, charm -> 6, decoration -> 7
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][0], 16), 0.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][1], 16), 1.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][2], 16), 2.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][3], 16), 3.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][4], 16), 4.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][5], 16), 5.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][6], 16), 6.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][7], 16), 7.0))

saver = tf.train.Saver()

W0820 11:28:43.318120  2396 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:94: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

W0820 11:28:43.329096  2396 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:122: The name tf.log is deprecated. Please use tf.math.log instead.

W0820 11:28:43.384026  2396 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:158: The name tf.train.exponential_decay is deprecated. Please use tf.compat.v1.train.exponential_decay instead.

W0820 11:28:43.402991  2396 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:159: The name tf.train.MomentumOptimizer is deprecated. Please use tf.compat.v1.train.MomentumOptimizer instead.



## restore model 생성

In [6]:
if not os.path.isfile(SAVE_FOLDER3 + 'model_v3_4.ckpt.index'):
    print('Create new model')
    sess.run(tf.global_variables_initializer())
    print('OK')
else:
    print('Restoring existed model')
    saver.restore(sess, SAVE_FOLDER3 + 'model_v3_4.ckpt')
    print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary/")

learning_rate = tf.get_collection('learning_rate')[0]

Create new model
OK


## epoch 에 따라서 training 수행

In [7]:
current_epoch = (int)(global_step.eval() / (len(real_train_data) // BATCH_SIZE))
for epoch in range(current_epoch + 1, NUM_EPOCHS):
    print('Epoch:', str(epoch))
    np.random.shuffle(real_train_data)
    train_img = []
    train_label = []
    train_mask = []

    for i in range(len(real_train_data)):
        train_img.append(real_train_data[i][0])
        train_label.append(real_train_data[i][1])
        train_mask.append(real_train_data[i][2])

    number_batch = len(real_train_data) // BATCH_SIZE

    avg_ttl = []
    avg_rgl = []
    avg_color_loss = []
    avg_shape_loss = []
    avg_opening_loss = []
    avg_strap_loss = []
    avg_pattern_loss = []
    avg_carrying_loss = []
    avg_charm_loss = []
    avg_decoration_loss = []

    color_nb_true_pred = 0
    shape_nb_true_pred = 0
    opening_nb_true_pred = 0
    strap_nb_true_pred = 0
    pattern_nb_true_pred = 0
    carrying_nb_true_pred = 0
    charm_nb_true_pred = 0
    decoration_nb_true_pred = 0

    color_nb_train = 0
    shape_nb_train = 0
    opening_nb_train = 0
    strap_nb_train = 0
    pattern_nb_train = 0
    carrying_nb_train = 0
    charm_nb_train = 0
    decoration_nb_train = 0

    print("Learning rate: %f" % learning_rate.eval())
    for batch in range(number_batch):
        # print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
        top = batch * BATCH_SIZE
        bot = min((batch + 1) * BATCH_SIZE, len(real_train_data))
        batch_img = np.asarray(train_img[top:bot])
        batch_label = np.asarray(train_label[top:bot])
        batch_mask = np.asarray(train_mask[top:bot])
        

        for i in range(BATCH_SIZE):
            if batch_mask[i] == 0.0:
                color_nb_train += 1
            else:
                if batch_mask[i] == 1.0:
                    shape_nb_train += 1
                else:
                    if batch_mask[i] == 2.0:
                        opening_nb_train += 1
                    else:
                        if batch_mask[i] == 3.0:
                            strap_nb_train += 1
                        else:
                            if batch_mask[i] == 4.0:
                                pattern_nb_train += 1
                            else:
                                if batch_mask[i] == 5.0:
                                    carrying_nb_train += 1
                                else:
                                    if batch_mask[i] == 6.0:
                                        charm_nb_train += 1
                                    else:
                                        decoration_nb_train +=1

        batch_img = augmentation(batch_img, 64)

        ttl, colorl, shapel, openingl, strapl, patternl, carryingl, charml, decol, l2l, _ = sess.run([total_loss, color_loss, shape_loss, opening_loss, strap_loss,
                                                                                      pattern_loss, carrying_loss, charm_loss, decoration_loss, l2_loss, train_step],
                                                                                      feed_dict={x: batch_img, y: batch_label, mask: batch_mask, is_training: True, keep_prob: DROP_OUT_PROB})

        color_nb_true_pred += sess.run(color_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True,keep_prob: DROP_OUT_PROB})

        shape_nb_true_pred += sess.run(shape_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        opening_nb_true_pred += sess.run(opening_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        strap_nb_true_pred += sess.run(strap_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        pattern_nb_true_pred += sess.run(pattern_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        carrying_nb_true_pred += sess.run(carrying_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        charm_nb_true_pred += sess.run(charm_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob:DROP_OUT_PROB})

        decoration_nb_true_pred += sess.run(decoration_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        avg_ttl.append(ttl)
        avg_color_loss.append(colorl)
        avg_shape_loss.append(shapel)
        avg_opening_loss.append(openingl)
        avg_strap_loss.append(strapl)
        avg_pattern_loss.append(patternl)
        avg_carrying_loss.append(carryingl)
        avg_charm_loss.append(charml)
        avg_decoration_loss.append(decol)

        avg_rgl.append(l2l)

    color_train_accuracy = color_nb_true_pred * 1.0 / color_nb_train
    shape_train_accuracy = shape_nb_true_pred * 1.0 / shape_nb_train
    opening_train_accuracy = opening_nb_true_pred * 1.0 / opening_nb_train
    strap_train_accuracy = strap_nb_true_pred * 1.0 / strap_nb_train
    pattern_train_accuracy = pattern_nb_true_pred * 1.0 / pattern_nb_train
    carrying_train_accuracy = carrying_nb_true_pred * 1.0 / carrying_nb_train
    charm_train_accuracy = charm_nb_true_pred * 1.0 / charm_nb_train
    decoration_train_accuracy = decoration_nb_true_pred * 1.0 / decoration_nb_train

    avg_color_loss = np.average(avg_color_loss)
    avg_shape_loss = np.average(avg_shape_loss)
    avg_opening_loss = np.average(avg_opening_loss)
    avg_strap_loss = np.average(avg_strap_loss)
    avg_pattern_loss = np.average(avg_pattern_loss)
    avg_carrying_loss = np.average(avg_carrying_loss)
    avg_charm_loss = np.average(avg_charm_loss)
    avg_decoration_loss = np.average(avg_decoration_loss)

    avg_rgl = np.average(avg_rgl)
    avg_ttl = np.average(avg_ttl)

    #     print('Avg_ttl: ' + str(avg_ttl))
    #     print('loss_summary_placeholder: ' + str(loss_summary_placeholder))
    #     print('merge_summary: ' + str(merge_summary))
    
    # 밑의 두 줄 error 발생해서 주석처리 해놓았음. TODO!
    
    #summary = sess.run(merge_summary, feed_dict={loss_summary_placeholder: avg_ttl})
    #writer.add_summary(summary, global_step=epoch)

    with open('log.csv', 'w+') as f:
        # epochs, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
        # avg_color_loss, avg_shape_loss, avg_opening_loss, avg_ttl, avg_rgl
        f.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14}, {15}, {16}, {17}, {18}\n'.format(current_epoch, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
                                                                                            strap_train_accuracy, pattern_train_accuracy, carrying_train_accuracy, charm_train_accuracy, decoration_train_accuracy,
                                                                                            avg_color_loss, avg_shape_loss, avg_opening_loss, avg_strap_loss, avg_pattern_loss,
                                                                                            avg_carrying_loss, avg_charm_loss, avg_decoration_loss, avg_ttl, avg_rgl))

    print('color task train accuracy: ' + str(color_train_accuracy * 100))
    print('shape task train accuracy: ' + str(shape_train_accuracy * 100))
    print('opening task train accuracy: ' + str(opening_train_accuracy * 100))
    print('strap task train accuracy: ' + str(strap_train_accuracy * 100))
    print('pattern task train accuracy: ' + str(pattern_train_accuracy * 100))
    print('carrying task train accuracy: ' + str(carrying_train_accuracy * 100))
    print('charm task train accuracy: ' + str(charm_train_accuracy * 100))
    print('decoration task train accuracy: ' + str(decoration_train_accuracy * 100))
    
    print('\n')

    print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
    print('Color loss: ' + str(avg_color_loss))
    print('Shape loss: ' + str(avg_shape_loss))
    print('opening loss: ' + str(avg_opening_loss))
    print('strap loss: ' + str(avg_strap_loss))
    print('pattern loss: ' + str(avg_pattern_loss))
    print('carrying loss: ' + str(avg_carrying_loss))
    print('charm loss: ' + str(avg_charm_loss))
    print('decoration loss: ' + str(avg_decoration_loss))

    print('\n')

    saver.save(sess, SAVE_FOLDER3 + 'model_v3_4.ckpt')

Epoch: 1
Learning rate: 0.010000
color task train accuracy: 29.162462159434916
shape task train accuracy: 64.53400503778337
opening task train accuracy: 33.65287588294652
strap task train accuracy: 65.91137965760322
pattern task train accuracy: 75.40322580645162
carrying task train accuracy: 44.008056394763344
charm task train accuracy: 43.087790110998995
decoration task train accuracy: 84.48362720403023


Total loss: 11.5236635. L2-loss: 3.8901322
Color loss: 1.9363292
Shape loss: 1.1347915
opening loss: 0.009352059
strap loss: 0.7680997
pattern loss: 0.37534758
carrying loss: 1.3217479
charm loss: 1.3313483
decoration loss: 0.7565148


Epoch: 2
Learning rate: 0.010000
color task train accuracy: 32.6448362720403
shape task train accuracy: 64.1633064516129
opening task train accuracy: 33.68580060422961
strap task train accuracy: 66.43109540636041
pattern task train accuracy: 86.93898134140191
carrying task train accuracy: 48.81492687846697
charm task train accuracy: 47.95969773299748
d

Epoch: 14
Learning rate: 0.008574
color task train accuracy: 48.08274470232089
shape task train accuracy: 68.11083123425693
opening task train accuracy: 33.702770780856426
strap task train accuracy: 66.96923852748361
pattern task train accuracy: 84.32459677419355
carrying task train accuracy: 62.178517397882
charm task train accuracy: 62.11586901763224
decoration task train accuracy: 84.58438287153652


Total loss: 6.5399885. L2-loss: 0.80735534
Color loss: 1.3610938
Shape loss: 0.9460956
opening loss: 0.0018582117
strap loss: 0.56124634
pattern loss: 0.3132861
carrying loss: 0.9707175
charm loss: 0.9823138
decoration loss: 0.5960215


Epoch: 15
Learning rate: 0.008574
color task train accuracy: 47.3551637279597
shape task train accuracy: 67.59072580645162
opening task train accuracy: 33.68633383761977
strap task train accuracy: 67.3902069661787
pattern task train accuracy: 83.42569269521411
carrying task train accuracy: 61.54233870967742
charm task train accuracy: 61.30982367758187
de

Epoch: 27
Learning rate: 0.007351
color task train accuracy: 48.79032258064516
shape task train accuracy: 68.816120906801
opening task train accuracy: 33.720343260979305
strap task train accuracy: 68.33081190115986
pattern task train accuracy: 76.04639435199194
carrying task train accuracy: 65.7272269753397
charm task train accuracy: 65.7258064516129
decoration task train accuracy: 84.43324937027708


Total loss: 6.414297. L2-loss: 0.9518622
Color loss: 1.3141578
Shape loss: 0.9057825
opening loss: 0.0010396783
strap loss: 0.5399502
pattern loss: 0.30280477
carrying loss: 0.90581775
charm loss: 0.8965265
decoration loss: 0.5963556


Epoch: 28
Learning rate: 0.007351
color task train accuracy: 49.11749873928392
shape task train accuracy: 70.35732259687973
opening task train accuracy: 33.60201511335013
strap task train accuracy: 69.1376701966717
pattern task train accuracy: 76.82619647355163
carrying task train accuracy: 67.92738275340393
charm task train accuracy: 66.26323751891074
deco

Epoch: 40
Learning rate: 0.006302
color task train accuracy: 51.40986908358509
shape task train accuracy: 70.73047858942066
opening task train accuracy: 33.66935483870967
strap task train accuracy: 69.92936427850655
pattern task train accuracy: 75.1008064516129
carrying task train accuracy: 68.76574307304786
charm task train accuracy: 70.21706208985361
decoration task train accuracy: 84.43324937027708


Total loss: 6.327425. L2-loss: 1.0560633
Color loss: 1.2719921
Shape loss: 0.8754372
opening loss: 0.00067797804
strap loss: 0.5187616
pattern loss: 0.30017814
carrying loss: 0.86120063
charm loss: 0.8597684
decoration loss: 0.5833455


Epoch: 41
Learning rate: 0.006302
color task train accuracy: 53.80736258194655
shape task train accuracy: 71.23425692695214
opening task train accuracy: 33.68633383761977
strap task train accuracy: 70.04538577912254
pattern task train accuracy: 73.81671701913393
carrying task train accuracy: 71.16935483870968
charm task train accuracy: 71.13350125944584


Epoch: 53
Learning rate: 0.005404
color task train accuracy: 52.62096774193549
shape task train accuracy: 72.78225806451613
opening task train accuracy: 33.618951612903224
strap task train accuracy: 70.4133064516129
pattern task train accuracy: 76.59788626069451
carrying task train accuracy: 74.76022211004543
charm task train accuracy: 74.67204843592332
decoration task train accuracy: 84.18932527693856


Total loss: 6.095486. L2-loss: 1.1804253
Color loss: 1.2150158
Shape loss: 0.79245967
opening loss: 0.00049826584
strap loss: 0.47190875
pattern loss: 0.30313885
carrying loss: 0.76647216
charm loss: 0.7637264
decoration loss: 0.6018411


Epoch: 54
Learning rate: 0.005133
color task train accuracy: 51.26008064516129
shape task train accuracy: 73.57539082198689
opening task train accuracy: 33.65239294710328
strap task train accuracy: 71.57258064516128
pattern task train accuracy: 77.25668179525971
carrying task train accuracy: 72.58064516129032
charm task train accuracy: 72.745591939546

...

Epoch: 92
Learning rate: 0.003235
color task train accuracy: 62.355309511826874
shape task train accuracy: 83.01411290322581
opening task train accuracy: 33.60242179616549
strap task train accuracy: 75.40322580645162
pattern task train accuracy: 72.90615539858727
carrying task train accuracy: 83.19878910191726
charm task train accuracy: 83.11491935483872
decoration task train accuracy: 85.85807750377454


Total loss: 5.6648974. L2-loss: 1.6985247
Color loss: 1.005953
Shape loss: 0.60542977
opening loss: 0.00030869318
strap loss: 0.37752897
pattern loss: 0.23682609
carrying loss: 0.6114992
charm loss: 0.60615563
decoration loss: 0.52267146


Epoch: 93
Learning rate: 0.003235
color task train accuracy: 62.31738035264483
shape task train accuracy: 82.45967741935483
opening task train accuracy: 33.635905194150276
strap task train accuracy: 76.13293051359517
pattern task train accuracy: 71.11895161290323
carrying task train accuracy: 81.91435768261964
charm task train accuracy: 82.047402924

Epoch: 105
Learning rate: 0.002774
color task train accuracy: 67.2544080604534
shape task train accuracy: 85.73588709677419
opening task train accuracy: 33.71975806451613
strap task train accuracy: 77.3851590106007
pattern task train accuracy: 70.94662638469285
carrying task train accuracy: 85.2896725440806
charm task train accuracy: 85.20949015648662
decoration task train accuracy: 87.41188318227593


Total loss: 5.697636. L2-loss: 2.001468
Color loss: 0.8873128
Shape loss: 0.5444902
opening loss: 0.0002927402
strap loss: 0.3025371
pattern loss: 0.24607457
carrying loss: 0.6257352
charm loss: 0.6351144
decoration loss: 0.454611


Epoch: 106
Learning rate: 0.002635
color task train accuracy: 65.7258064516129
shape task train accuracy: 84.61150353178607
opening task train accuracy: 33.65239294710328
strap task train accuracy: 78.07459677419355
pattern task train accuracy: 66.70025188916877
carrying task train accuracy: 86.59274193548387
charm task train accuracy: 86.68683812405446
decor

Epoch: 118
Learning rate: 0.002259
color task train accuracy: 69.95967741935483
shape task train accuracy: 89.70736629667003
opening task train accuracy: 33.60201511335013
strap task train accuracy: 80.23197175995965
pattern task train accuracy: 68.56423173803526
carrying task train accuracy: 91.53225806451613
charm task train accuracy: 90.87701612903226
decoration task train accuracy: 89.72292191435768


Total loss: 5.219066. L2-loss: 2.2667575
Color loss: 0.7766214
Shape loss: 0.46007007
opening loss: 0.00027511394
strap loss: 0.24844785
pattern loss: 0.19235228
carrying loss: 0.44802082
charm loss: 0.4514719
decoration loss: 0.37504894


Epoch: 119
Learning rate: 0.002259
color task train accuracy: 71.10438729198184
shape task train accuracy: 89.92950654582074
opening task train accuracy: 33.618951612903224
strap task train accuracy: 80.36253776435045
pattern task train accuracy: 64.66431095406361
carrying task train accuracy: 90.8219868885527
charm task train accuracy: 90.922844175

Epoch: 131
Learning rate: 0.001937
color task train accuracy: 75.46693589096417
shape task train accuracy: 92.03227433182047
opening task train accuracy: 33.68580060422961
strap task train accuracy: 81.41057934508817
pattern task train accuracy: 66.09485368314833
carrying task train accuracy: 92.68415741675075
charm task train accuracy: 92.24962254655259
decoration task train accuracy: 93.30312185297079


Total loss: 5.0564704. L2-loss: 2.4899197
Color loss: 0.7067706
Shape loss: 0.3686228
opening loss: 0.00025923387
strap loss: 0.22921483
pattern loss: 0.15975201
carrying loss: 0.40476185
charm loss: 0.39733246
decoration loss: 0.2998368


Epoch: 132
Learning rate: 0.001937
color task train accuracy: 75.88294651866802
shape task train accuracy: 92.24572004028198
opening task train accuracy: 33.68633383761977
strap task train accuracy: 82.51008064516128
pattern task train accuracy: 65.70996978851964
carrying task train accuracy: 93.44758064516128
charm task train accuracy: 93.488137304

Epoch: 144
Learning rate: 0.001661
color task train accuracy: 79.08266129032258
shape task train accuracy: 94.662638469285
opening task train accuracy: 33.66884750880725
strap task train accuracy: 82.34997478567827
pattern task train accuracy: 68.09475806451613
carrying task train accuracy: 94.85627836611195
charm task train accuracy: 94.70499243570349
decoration task train accuracy: 94.90413723511605


Total loss: 4.752522. L2-loss: 2.6038952
Color loss: 0.5463636
Shape loss: 0.31772682
opening loss: 0.00024206625
strap loss: 0.17783806
pattern loss: 0.14936912
carrying loss: 0.3498421
charm loss: 0.34061906
decoration loss: 0.2666258


Epoch: 145
Learning rate: 0.001661
color task train accuracy: 78.89168765743074
shape task train accuracy: 95.11826874685455
opening task train accuracy: 33.60242179616549
strap task train accuracy: 82.82115869017632
pattern task train accuracy: 67.65743073047858
carrying task train accuracy: 95.56675062972292
charm task train accuracy: 95.862764883955